In [1]:
from preprocess_raw import *
from preprocess_transformed import *
from model_setup_fit import *
from model_optimization import *
import warnings
warnings.filterwarnings("ignore")
pd.get_option("display.max_rows",999)
pd.get_option("display.max_columns",999)
pd.get_option("display.width",None)

GITHUB = os.environ.get("GITHUB")
sys.path.insert(0, GITHUB + 'Machine_Cleaning_2019/modeling/src')

HOST_FORKED = os.environ.get("HOST_FORKED")
USER_FORKED = os.environ.get("USER_FORKED")
PASSWORD_FORKED = os.environ.get("PASSWORD_FORKED")
DB_FORKED = os.environ.get("DB_FORKED")

HOST_DAR = os.environ.get("HOST_DAR")
USER_DAR = os.environ.get("USER_DAR")
PASSWORD_DAR = os.environ.get("PASSWORD_DAR")
DB_DAR = os.environ.get("DB_DAR")
   
## Function to load in a model and features and apply it to new data. 
    # Outputs a data frame with 2 columns: line_item_id (pristine table), predicted variable
    # Must input the data frame to predict on (after the minimal preprocessing) and a model id (string)    
def load_and_predict(data, model_id):
    list_unpickle = open('objects/feature_cols/features_' + model_id + '.pkl', 'r')
    # load the unpickle object into a variable
    model_cols = load(list_unpickle)
    print len(model_cols)

    # make data frame from dm.ml_model_results_lookup for the model
    model_results_lookup = get_table_from_db("select * from dm.ml_model_results_lookup where model_id = '" + model_id + "';", 'string', 
                                              HOST_FORKED, USER_FORKED, PASSWORD_FORKED, DB_FORKED)
    yvar = model_results_lookup['y'].item() 
        
    # load model
    model = load(open('objects/models/model_' + model_id + '.pkl', 'rb'))
    print model
    # create imputer (based on output table)
    imputer = Imputer(strategy=model_results_lookup['imputer'].item())

    # subset to particular model's columns
    data_model = data[model_cols]
    print data_model.shape
    # impute missing values
    data_model = imputer.fit_transform(data_model)
    # predict
    y_pred = model.predict(data_model) 
    # need to add predict_proba()
    y_pred_proba = model.predict_proba(data_model)
    df_pred_probab = pd.DataFrame(y_pred_proba).reset_index(drop=True)

    
    # Get label encoder for y variable (for classification)
    if yvar != 'fiber_binary' and yvar != 'num_lines' and yvar != 'exclude':
        yquery = get_clean_y_query(yvar,'get_yvar_dar_prod.sql')
        
        yclean = get_table_from_db(yquery, 'string', HOST_DAR, USER_DAR, PASSWORD_DAR, DB_DAR)
        le = preprocessing.LabelEncoder()
        
        if yvar == 'connect_category':
            labels = np.append(yclean[yvar],'Uncategorized')
            le.fit(labels)
        else:
            le.fit(yclean[yvar])

        # get pretty names
        y_pred = le.inverse_transform(y_pred)

    # concatenate
    line_item_id = pd.Series(data['frn_adjusted']).reset_index(drop=True) ## WILL BE DIFFERENT
    y_pred = pd.Series(y_pred).reset_index(drop=True)
    output = pd.concat([line_item_id, y_pred, df_pred_probab],axis=1)
    output.columns = ['line_item_id','prediction'] + le.classes_.tolist()
    
    # print category diffs
    if yvar != 'fiber_binary' and yvar != 'num_lines' and yvar != 'exclude':
        y_counts = yclean[yvar].value_counts(normalize=True).reset_index()
        ypred_counts = output.prediction.value_counts(normalize=True).reset_index()
        compdf = y_counts.merge(ypred_counts,on='index')
        compdf['absdiff'] = abs(compdf[yvar] - compdf['prediction'])
        print "y-variable distributions comparison:"
        print compdf
    else:
        print output.prediction.value_counts()
        
    return output 

In [2]:
# replace with get_data_future_predict.sql!!! also, credentials will probably be different
raw_data_predict = get_table_from_db('get_data_2019_train.sql', 'file', 
                                     HOST=HOST_FORKED,
                                     USER=USER_FORKED,
                                     PASSWORD=PASSWORD_FORKED, 
                                     DB=DB_FORKED
                                    )

Querying data from DB connection
Trying to establish initial connection to the server
Success!
Finished querying data


In [3]:
print(raw_data_predict.shape)
print(raw_data_predict.columns.sort_values().tolist())

(47165, 118)
['account_number', 'actual_start_date', 'annual_interest_rate', 'appeal_wave_number', 'application_number', 'average_cost_per_foot_of_outside_plant', 'award_date', 'baloon_payment', 'based_on_multiple_award_schedule', 'based_on_state_master_contract', 'basic_firewall_protection', 'ben', 'billed_entity_name', 'billed_entity_type', 'burstable_speed', 'burstable_speed_units', 'connected_directly_to_school_library_or_nif', 'connection_supports_school_library_or_nif', 'connection_used_by', 'contact_email', 'contract_end_time', 'contract_expiration_date', 'contract_number', 'contract_type', 'created_at', 'discount_rate', 'download_speed', 'download_speed_mbps', 'download_speed_units', 'establishing_fcc_form470', 'extended_contract_expiration_date', 'fcc_form486', 'fcc_form486_case_status', 'fcc_form486_invoicing_ready', 'fcdl_comment_app', 'fcdl_comment_frn', 'fcdl_letter_date', 'fiber_sub_type', 'fiber_type', 'form_version', 'frn', 'frn_adjusted', 'frn_number_from_the_previous_

In [4]:
print("*** STARTING PRE-PROCESSING FOR RAW DATA ***")
clean_data_predict = PreprocessRaw(raw_data_predict, verbose=True)
clean_data_predict = clean_data_predict.applyall_predict()

data_2019 = clean_data_predict.getdata()

print data_2019.shape

*** STARTING PRE-PROCESSING FOR RAW DATA ***
Dropped 0 duplicate rows
Dropped null columns: 
['actual_start_date', 'fiber_type', 'fiber_sub_type', 'total_project_plant_route_feet', 'average_cost_per_foot_of_outside_plant', 'total_strands', 'number_of_erate_eligible_strands', 'match_amount', 'source_of_matching_funds', 'total_amount_financed', 'total_number_of_terms_in_months', 'annual_interest_rate', 'baloon_payment', 'special_construction_state_tribal_match_percentage', 'pending_reason', 'fcc_form486', 'fcc_form486_case_status', 'fcc_form486_invoicing_ready', 'last_date_to_invoice', 'wave_sequence_number', 'fcdl_letter_date', 'user_generated_fcdl_date', 'fcdl_comment_app', 'fcdl_comment_frn', 'appeal_wave_number', 'revised_fcdl_date', 'invoicing_mode', 'total_authorized_disbursement', 'connection_used_by', 'make', 'model', 'other_manufacture', 'unit']
months_of_service duplicate with total_number_of_months_of_service, dropping months_of_service
lease_or_non_purchase_agreement duplicat

In [5]:
### Individual Model Predictions

## Purpose
purpose_predictions = load_and_predict(data_2019, '1311') # your model_id from dl.ml_model_results_lookup

54
Querying data from DB connection
Trying to establish initial connection to the server
Success!
Finished querying data
RandomForestClassifier(bootstrap=False, class_weight='balanced',
            criterion='gini', max_depth=30, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=5, min_weight_fraction_leaf=0.0,
            n_estimators=350, n_jobs=-1, oob_score=False, random_state=7,
            verbose=0, warm_start=False)
(47165, 54)
Querying data from DB connection
Trying to establish initial connection to the server
Success!
Finished querying data
y-variable distributions comparison:
      index   purpose  prediction   absdiff
0  internet  0.457153    0.563066  0.105912
1       wan  0.354876    0.273868  0.081007
2  upstream  0.131290    0.116930  0.014361
3       isp  0.049102    0.037231  0.011871
4  backbone  0.007579    0.008905  0.001326


In [6]:
print("purpose prediction columns: ")
print(purpose_predictions.columns.tolist())

purpose prediction columns: 
['line_item_id', 'prediction', 'backbone', 'internet', 'isp', 'upstream', 'wan']


In [7]:
## add back frn_complete and original info
ml_mass_update_qa_prf = pd.merge(raw_data_predict[['frn_adjusted','purpose', 'function']], purpose_predictions, left_on='frn_adjusted', right_on='line_item_id')

In [8]:
### Load Mappings, convert easy ones to dicts

# connect_type_different = pd.read_csv('../data/connect_type_different.csv')
# connect_type_different = connect_type_different.set_index('Connect Category ')
# connect_type_different_dict = connect_type_different.to_dict()
# connect_type_different_dict = connect_type_different_dict['Connect Type']

# connect_type_same = pd.read_csv('../data/connect_type_same.csv')

# function = pd.read_csv('../data/function.csv')
# function = function.set_index('Connect Category ')
# function_dict = function.to_dict()
# function_dict = function_dict['Function']

In [9]:
##### Where model updated cc

# model_1_cc_different = ml_mass_update_qa_final[ml_mass_update_qa_final.prediction_connect_category!=ml_mass_update_qa_final.connect_category]
# model_1_cc_different['prediction_connect_type'] = model_1_cc_different['prediction_connect_category'].map(connect_type_different_dict)

# ##remove predictions where we're changing a T-1 or fractional T-1 or ISDN BRI to be Fiber
# model_1_cc_different = model_1_cc_different[~(((model_1_cc_different['connect_type']=='T-1')
#                                                |(model_1_cc_different['connect_type']=='Fractional T-1')
#                                                |(model_1_cc_different['connect_type']=='ISDN-BRI'))
#                                               &((model_1_cc_different['prediction_connect_category']=='Lit Fiber')
#                                                 |(model_1_cc_different['prediction_connect_category']=='Dark Fiber')))]

# ##### Where model didn't update cc

# model_1_cc_same = ml_mass_update_qa_final[ml_mass_update_qa_final.prediction_connect_category==ml_mass_update_qa_final.connect_category]
# model_1_cc_same_cts = model_1_cc_same[['line_item_id','prediction_connect_category','connect_type']].merge(connect_type_same,left_on=['prediction_connect_category','connect_type'],right_on=['Connect Category ','Original Connect Type'],how='left')
# model_1_cc_same_cts['prediction_connect_type'] = np.where(model_1_cc_same_cts['Update CT']==1,model_1_cc_same_cts['Connect Type'],model_1_cc_same_cts['connect_type'])

# # filter for the 3 columns
# model_1_cc_same = model_1_cc_same.merge(model_1_cc_same_cts[['line_item_id','prediction_connect_type']],on='line_item_id')
# # union back
# model_1_final = pd.concat([model_1_cc_different,model_1_cc_same]) 

# ##### Function update

# model_1_final['prediction_function'] = model_1_final['prediction_connect_category'].map(function_dict)

# # QA
# print(model_1_final[['line_item_id','prediction_connect_category','prediction_connect_type']].groupby(['prediction_connect_category','prediction_connect_type']).count())
# print(model_1_final[['line_item_id','prediction_connect_category','prediction_function']].groupby(['prediction_connect_category','prediction_function']).count())

In [10]:
###### everywhere the model(s) predict purpose = ISP, _don’t update connect type or function_

# model_1_final['prediction_connect_type'] = np.where(model_1_final['prediction_purpose']=='isp',None,model_1_final['prediction_connect_type'])
# model_1_final['prediction_function'] = np.where(model_1_final['prediction_purpose']=='isp',None,model_1_final['prediction_function'])

# model_1_final.isnull().sum()

### csv for QA, eventual mass update in environment

In [11]:
ml_mass_update_qa_prf.to_csv('../data/ml_mass_update_qa_prf.csv',index=False)

In [12]:
ml_mass_update_qa_prf.shape

(47165, 10)